In [3]:
import os
import matplotlib.pyplot as plt

from lvmdrp.utils.examples import get_frames_metadata

from lvmdrp.functions import imageMethod
from lvmdrp.functions import rssMethod

plt.style.use("seaborn-v0_8-talk")

In [4]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = os.path.join("data")
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.{ext}")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.{ext}")

# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "sim_2d"))
# group by imagetyp, camera and exptime
frames_table = frames_table.group_by(["imagetyp", "spec", "camera", "expnum", "exptime"])
# define groups and unique categories in those groups
frames_groups = frames_table.groups
unique_groups = frames_groups.keys

[INFO]: loading cached metadata from '/home/sanchez/sas/sdsswork/lvm/sandbox/lvm_metadata_None.pkl'


IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
# expected number of fibers per spec 648

fiberflats = frames_groups[(unique_groups["imagetyp"]=="flat")&(unique_groups["spec"]=="sp1")&(unique_groups["expnum"]=="00001002")]
fiberflats.pprint()

for fiberflat_frame in fiberflats:
    fiberflat_path = fiberflat_frame["path"]
    fiberflat_camera, fiberflat_expnum = fiberflat_frame["camera"], fiberflat_frame["expnum"]

    imageMethod.findPeaksAuto_drp(
        in_image=out_main_path.format(kind="cfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        out_peaks=out_main_path.format(kind="peaks", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="txt"),
        nfibers=648, slice=1870, threshold=1, method="hyperbolic", median_box=5, median_cross=1, plot=1
    )

    imageMethod.tracePeaks_drp(
        in_image=out_main_path.format(kind="cfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        out_trace=out_main_path.format(kind="trace", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        in_peaks=out_main_path.format(kind="peaks", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="txt"),
        steps=30, coadd=30, threshold_peak=1, max_diff=2, method="hyperbolic", median_box=5, median_cross=1,
        poly_disp=2, plot=1
    )

    imageMethod.extractSpec_drp(
        in_image=out_main_path.format(kind="cfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        out_rss=out_main_path.format(kind="xfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        in_trace=out_main_path.format(kind="trace", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        method="aperture", aperture=5, plot=1, parallel="auto"
    )

NameError: name 'frames_groups' is not defined